<a href="https://colab.research.google.com/github/ZiyueNie/19ma573ZiyueNie/blob/master/src/hw7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1**.Verify the explicit form of $r_{t}$ given by

$$r_t = r_0e^{-\kappa t} + \mu(1 - e^{-\kappa t}) + \sigma e^{-\kappa t}\int_{0}^{t}e^{\kappa s} dW_s$$ is the solution to the original vasicek model
$$
dr_t = \kappa(\mu - r_t)dt + \sigma dW_t
$$
<br>**Proof**
<br>
$$
\begin{align*}
&dr_t = \kappa(\mu - r_t)dt + \sigma dW_t \\
&dr_t+\kappa r_t dt=\kappa \mu dt+\sigma dW_t\\
&e^{\kappa t}dr_t+ e^{\kappa t}\kappa r_t dt=e^{\kappa t}\kappa \mu dt+e^{\kappa t}\sigma dW_t\\
&d(e^{\kappa t}r_t)=e^{\kappa t}\kappa \mu dt+e^{\kappa t}\sigma dW_t\\
&\int_0^t d(e^{\kappa t}r_t)=\mu\int_0^t \kappa e^{\kappa t}  dt +\sigma\int_0^t e^{\kappa s}dW_s \\
&e^{\kappa t}r_t-r_0=\mu(e^{\kappa t}-1)+\sigma\int_0^t e^{\kappa s}dW_s \\
&r_t = r_0e^{-\kappa t} + \mu(1 - e^{-\kappa t}) + \sigma e^{-\kappa t}\int_{0}^{t}e^{\kappa s} dW_s
\end{align*}
$$

**2.** Design pricing engine of ZCB P(0,T) using explicit form
$$r_t = r_0e^{-\kappa t} + \mu(1 - e^{-\kappa t}) + \sigma e^{-\kappa t}\int_{0}^{t}e^{\kappa s} dW_s$$

In [0]:
import numpy as np
import scipy.stats as ss
import scipy.optimize as so
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
def zcb_explicit(T,kappa,sigma,mu,r0):
  B=(1-np.exp(-kappa*(T)))/kappa
  A=(mu-sigma**2/(2*kappa**2))*(B-T)-sigma**2/(4*kappa)*(B**2)
  return np.exp(A-B*r0)

**3.** Design alternative pricing engine of ZCB P(0,T) using exact sampling

In [0]:
def zcb_exactsampling(T,kappa,mu,sigma,r0,n):
  mu=(mu*T)+((r0-mu)*(1-np.exp(-kappa*T))/kappa)
  var=((sigma**2)/(2*(kappa**3)))*((2*kappa*T)-3+(4*np.exp(-kappa*T))-np.exp(-2*kappa*T))
  r= np.random.normal(mu,var,n)
  r_=np.exp(-r)
  return np.mean(r_)

**4.** Compute ZCB P(0,1) Libor L(0,1) with the following parameters using above two different pricing engines.

In [0]:
theta=[0.1,0.05,0.003,0.03]
kappa,mu,sigma,r0=theta
T=1
n=100

In [16]:
P_explicit=zcb_explicit(T,kappa,sigma,mu,r0)
print('ZCB P(0,1) price using explicit form is '+ str(P_explicit))
P_exactsampling=zcb_exactsampling(T,kappa,mu,sigma,r0,n)
print('ZCB P(0,1) price using exact sampling is '+ str(P_exactsampling))

ZCB P(0,1) price using explicit form is 0.9695084475425054
ZCB P(0,1) price using exact sampling is 0.9695071856352797


In [0]:
def Libor(T,P):
  return 100/T*(1/P-1)
  

In [21]:
L_explicit=Libor(T,P_explicit)
print('ZCB L(0,1) price using explicit form is '+ str(L_explicit))
L_exactsampling=Libor(T,P_exactsampling)
print('ZCB L(0,1) price using exact sampling is '+ str(L_exactsampling))

ZCB L(0,1) price using explicit form is 3.145052787810565
ZCB L(0,1) price using exact sampling is 3.1452035092872643


**5.** Find 10 term swap rates with term length 1/2 year

In [0]:
P1=[]
P2=[]
def Swap(T,N,kappa,sigma,mu,r0):
  delta=T/N
  for j in range(N):
    P1.append(zcb_explicit((j+1)*delta,kappa,sigma,mu,r0)) # using explicit form
    P2.append(zcb_exactsampling((j+1)*delta,kappa,mu,sigma,r0,n)) # using exact sampling
  S1=100*(1-zcb_explicit(T,kappa,sigma,mu,r0))/(delta*sum(P1))
  S2=100*(1-zcb_exactsampling(T,kappa,mu,sigma,r0,n))/(delta*sum(P2))
  return S1,S2  

In [37]:
S1,S2=Swap(5,10,kappa,sigma,mu,r0)
print('10 term swap rates with term length 1/2 year using explicit form is '+ str(S1))
print('10 term swap rates with term length 1/2 year using exact sampling is '+ str(S2))


10 term swap rates with term length 1/2 year using explicit form is 3.441821396389877
10 term swap rates with term length 1/2 year using exact sampling is 3.4439531251932243


**6.** Pick a date, and using Libor market data of that data, calibrate Vasicek model. Then compare market rate and calibrated rate in a plot